# Lesson 2 : LangGraph Components

In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [6]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [7]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [8]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(len(tool_calls))
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [9]:
tool

TavilySearchResults(max_results=4)

In [10]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [ ]:
!pip show pygraphviz

In [ ]:
!pip install pygraphviz

In [ ]:
!which python

In [ ]:
from IPython.display import Image
import pygraphviz as pgv
Image(abot.graph.get_graph().draw_png())

In [11]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

1
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_20zFve3KGMZO1aXrP2RRTRat', 'type': 'tool_call'}
Back to the model!


In [12]:
result

{'messages': [HumanMessage(content='What is the weather in sf?'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_20zFve3KGMZO1aXrP2RRTRat', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 153, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3e855956-962c-4fa1-9a7a-7fedce786613-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_20zFve3KGMZO1aXrP2RRTRat', 'type': 'tool_call'}]),
  ToolMessage(content='[{\'url\': \'https://weathershogun.com/weather/usa/ca/san

In [13]:
result['messages'][-1].content

'The current weather in San Francisco is partly cloudy with a temperature of 53.1°F. The wind speed is 11.9 km/h coming from the WSW direction. The humidity is at 89% with a cloud cover of 75%.'

In [14]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

2
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_cVbsDKGZNIPavR6sKje1B7Gg', 'type': 'tool_call'}
2
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_qjo1kuGcn8CA9YEsmQSFEBdP', 'type': 'tool_call'}
Back to the model!


In [15]:
result['messages'][-1].content

'The weather in San Francisco is currently partly cloudy with a temperature of 53.1°F. The wind speed is 11.9 km/h coming from the WSW direction. The humidity is at 89%, and the visibility is 16.0 km.\n\nIn Los Angeles, the weather is clear with a temperature of 55.9°F. The wind speed is 5.4 km/h coming from the SSW direction. The humidity is at 80%, and there are no clouds in the sky.'

In [21]:
query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

In [22]:
messages

[HumanMessage(content='Who won the super bowl in 2024? In what state is the winning team headquarters located? What is the GDP of that state? Answer each question.')]

In [23]:
model = ChatOpenAI(model="gpt-4o")

In [24]:
abot = Agent(model, [tool], system=prompt)

In [25]:
result = abot.graph.invoke({"messages": messages})

1
Calling: {'name': 'tavily_search_results_json', 'args': {'query': '2024 Super Bowl winner'}, 'id': 'call_A7G6ujhhqgBTGVb7Y2zHSKYm', 'type': 'tool_call'}
Back to the model!
2
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Philadelphia Eagles headquarters location'}, 'id': 'call_qCSRtauPDzr4PWx9KtSk8XVm', 'type': 'tool_call'}
2
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Pennsylvania 2024'}, 'id': 'call_bQ6TGm27ecvVJkqCpmbuzSQS', 'type': 'tool_call'}
Back to the model!


In [26]:
result

{'messages': [HumanMessage(content='Who won the super bowl in 2024? In what state is the winning team headquarters located? What is the GDP of that state? Answer each question.'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_A7G6ujhhqgBTGVb7Y2zHSKYm', 'function': {'arguments': '{"query":"2024 Super Bowl winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 176, 'total_tokens': 199, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_92f14e8683', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c85d182e-d4b2-4e14-9317-1113abeec2d1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 Super Bowl winner'}, 'id': 'call_A7G6ujhhqgBTGVb7Y

In [20]:
result['messages'][-1].content

"1. **Who won the Super Bowl in 2024?**  \n   The Kansas City Chiefs won the Super Bowl in 2024, defeating the San Francisco 49ers 25-22 in overtime.\n\n2. **In what state is the winning team's headquarters located?**  \n   The headquarters of the Kansas City Chiefs is located in Kansas City, Missouri.\n\n3. **What is the GDP of that state?**  \n   Missouri's GDP was approximately $460.7 billion in the fourth quarter of 2024."

In [27]:
query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

1
Calling: {'name': 'tavily_search_results_json', 'args': {'query': '2024 Super Bowl winner'}, 'id': 'call_mX1hGkoOdt2TZKircdH2iSuZ', 'type': 'tool_call'}
Back to the model!
2
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Philadelphia Eagles headquarters location'}, 'id': 'call_hR30LVnrmvmHap5VcmgK0MkC', 'type': 'tool_call'}
2
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Pennsylvania 2024'}, 'id': 'call_losW7V7dCVcNws1VsWBLjSoH', 'type': 'tool_call'}
Back to the model!


In [28]:
print(result['messages'][-1].content)

1. **Who won the Super Bowl in 2024?**  
   The Philadelphia Eagles won the Super Bowl in 2024.

2. **In what state is the winning team headquarters located?**  
   The Philadelphia Eagles' headquarters is located in Philadelphia, Pennsylvania.

3. **What is the GDP of that state?**  
   The GDP of Pennsylvania in 2024 was approximately $1,024,206.3 million (or about $1.024 trillion).
